<a href="https://colab.research.google.com/github/juanpamoji/Repo1/blob/main/EntrenaGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entendiento Google Colab**
# **Juan pablo Mojica Macías**
# Diplomado en IA y AP


# **Ejemplo para GPU**

In [ ]:
import numpy as np
import os 
import gzip 

def datos (ruta, tipo="train"):
  ruta_categorias=os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes=os.path.join(ruta, '%s-images-idx3-ubyte.gz'% tipo)

  with gzip.open(ruta_categorias,'rb') as rut_cat:
    etiquetas=np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

  with gzip.open(ruta_imagenes,'rb') as rut_imgs:
    imagenes=np.frombuffer(rut_imgs.read(),dtype=np.uint8, offset=16).reshape(len(etiquetas),784)

  return imagenes, etiquetas

# **Lectura de datos en Drive**


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta='gdrive/My Drive/Trabajos Diplomado IA/fashion_mnist_data'

X_train, Y_train=datos(ruta,tipo='train')
X_test, Y_test=datos(ruta,tipo='test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Reajuste del tamaño de datos (Multiplo de 128)**

In [ ]:
n_train=len(X_train)
n_test=len(X_train)
mod_train=60000-(60000%128)
mod_test=10000-(10000%128)

X_train=X_train[0:mod_train,:]
X_test=X_test[0:mod_test,:]
Y_train=Y_train[0:mod_train]
Y_test=Y_test[0:mod_test]

X_train=np.reshape(X_train,(mod_train,28,28,1))
X_test=np.reshape(X_test,(mod_test,28,28,1))

# Importación de ***Tensorflow 2*** plus ***Keras***

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.4.1


# **Creación del Modelo con Keras**

In [ ]:
tf.random.set_seed(200)

model = tf.keras.models.Sequential() #Contenedor del modelo
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:])) #Normalización primera Capa
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu')) #Filtro convolucional
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2))) #Capa maxpooling
model.add(tf.keras.layers.Dropout(0.25)) #reduce el overfitting del modelo

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:])) #Normalización segunda  Capa
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu')) #Filtro convolucional
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) #Capa maxpooling
model.add(tf.keras.layers.Dropout(0.25)) #reduce el overfitting del modelo

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))#Normalización tercera Capa
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu')) #Filtro convolucional
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))#Capa maxpooling
model.add(tf.keras.layers.Dropout(0.25)) #reduce el overfitting del modelo

model.add(tf.keras.layers.Flatten()) #Aplana datos
model.add(tf.keras.layers.Dense(256)) #Agragamos una red neurona con 256 neuronas
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10)) #conexión de salida con 10 neuronas
model.add(tf.keras.layers.Activation('softmax'))

#Imprime el modelo 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)        

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# **Entrenamiento con GPU**

# Verificación de la disponibilidad de GPU

In [ ]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


In [ ]:
# Entrenamiento GPU
import timeit

def gpu_train():
  with tf.device('/GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

gpu_time = timeit.timeit('gpu_train()', number=1, setup='from __main__ import gpu_train')

Epoch 1/2
468/468 [==============================] - 11s 17ms/step - loss: 1.0176 - accuracy: 0.7070 - val_loss: 0.3877 - val_accuracy: 0.8660
Epoch 2/2
468/468 [==============================] - 8s 17ms/step - loss: 0.4055 - accuracy: 0.8566 - val_loss: 0.3381 - val_accuracy: 0.8859


In [ ]:
print('Tiempo de entrenamiento: ' + str(gpu_time) + ' segundos')

Tiempo de entrenamiento: 18.920414106999942 segundos


# **Nota:**

El entrenamiento con **GPU** se demoró aproximadamente 18.92 segundos 